In [1]:
from datetime import datetime
import firecloud.api as FAPI
import json
import os
import pandas as pd
import pprint
import re
import requests

# https://github.com/broadinstitute/firecloud-orchestration/blob/develop/src/main/resources/library/attribute-definitions.json
# https://github.com/broadinstitute/firecloud-orchestration/blob/219bdb1a7c4c85df5350f27be9292bb66233a355/src/main/resources/library/attribute-definitions.json

#  "consentCodes": {
#     "GRU": "For health/medical/biomedical purposes and other biological research, including the study of population origins or ancestry.",
#     "HMB": "Use of the data is limited to health/medical/biomedical purposes, does not include the study of population origins or ancestry.",
#     "DS": "Use of the data must be related to a particular disease.",
#     "NCU": "Use of the data is limited to non-commercial use.",
#     "NPU": "Use of the data is limited to not-for-profit organizations.",
#     "NMDS": "Use of the data includes methods development research (e.g., development of software or algorithms) ONLY within the bounds of other specified data use limitations.",
#     "NAGR": "Use of the data includes aggregate level analysis to address research questions (e.g., determine variant frequencies across the general population) ONLY within the bounds of other specified data use limitations.",
#     "NCTRL": "Data can be used as a control set ONLY within the bounds of other specified data use limitations (e.g. only for cancer studies).",
#     "RS-G": "Use of the data is limited to studies of particular gender.",
#     "RS-PD": "Use of the data is limited to pediatric research."

#   "required": ["library:datasetName", "library:datasetVersion", "library:datasetDescription", "library:datasetCustodian",
#     "library:datasetDepositor", "library:contactEmail", "library:datasetOwner", "library:institute", "library:indication",
#     "library:numSubjects", "library:projectName", "library:dataCategory", "library:datatype", "library:dataUseRestriction",
#     "library:studyDesign", "library:requiresExternalApproval", "library:useLimitationOption"],
#   "oneOf": [{
#     "required": ["library:GRU", "library:HMB", "library:NCU", "library:NPU", "library:NMDS", "library:NAGR",
#       "library:NCTRL", "library:RS-G", "library:RS-PD", "library:useLimitationOption", "library:IRB"],
#     "anyOf": [
#       {
#         "properties": {"library:GRU": {"enum": [true]}}
#       },
#       {
#         "properties": {"library:HMB": {"enum": [true]}}
#       },
#       {
#         "required": ["library:DS"],
#         "properties": {"library:DS": {"minItems": 1}}
#       }
#     ],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["questionnaire"]
#       }
#     }
#   }, {
#     "required": ["library:orsp", "library:useLimitationOption"],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["orsp"]
#       }
#     }
#   }, {
#     "required": ["library:useLimitationOption"],
#     "properties": {
#       "library:useLimitationOption": {
#         "type": "string",
#         "enum": ["skip"]
#       }
#     }
#   }],


print(FAPI.whoami())
pp = pprint.PrettyPrinter(indent=4)

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"
API_ROOT = "https://api.firecloud.org/api"


TEST_WORKSPACE = 'ANVIL_CMG_BROAD_BRAIN_ENGLE_WES'

def getDefaultAccessToken():
    result = !gcloud auth application-default print-access-token
    return result[0]


ACCESS_TOKEN = getDefaultAccessToken()




pet-117272931645288568532@terra-e36fcccd.iam.gserviceaccount.com


In [7]:
def getUpdateAttributesURL(namespace,workspace):
    return  API_ROOT+"/library/"+ namespace + "/" + workspace + "/metadata"

def getAPIHeaders():
    headers = {
        'accept': 'application/json',
        'Authorization' : 'Bearer '+ACCESS_TOKEN
    }
    return headers


# Replaces the set the tags on a workspace to the given list of tags
def updateAttributes(namespace,workspace,attributes):        
    url = getUpdateAttributesURL(namespace,workspace)
    headers = getAPIHeaders()
#     resp = requests.put(url, json=attributes, headers=headers)
    print(resp.status_code)
    print(resp.text)

# 
def getWorkspaceAttributes(namespace,workspace):        
    url = getUpdateAttributesURL(namespace,workspace)
    headers = getAPIHeaders()
    resp = requests.get(url, headers=headers)
    print(resp.status_code)
    print(json.dumps(resp.json()))

"""
For the catalog we want to set:

1.  consent code - how?
2. Library:indication -  "library:indication": "Poor health",
3. library:studyDesign - "library:studyDesign": "Case/Control"
4. library:dataType -  "library:datatype": ["Whole Exome", "Whole Genome"],

consortia tag
phsid tag

maybe make numSubjects match the workspace data


TODO: 


1. Combine different spreadsheets dave using + tags

1. Strip down input to only the columns we care about

1. Validate the spreadsheet contains only allowed values for study design and data type - nope just show wht the  have for now
and try to get to standardise maybe on values from TIM

DataType
Whole Exome, Whole Genome, Exome Panel, Genotyping Array, RNA-Seq, Methylation Array
     
Study Design
Case/Control, Tumor/Normal, Trio/Twin/Family, Prospective Longitudinal Cohort, Cases only - somatic, Cases only - germline, Controls
     
"""

# testAttributes = {
    
#     "library:datasetName": "Workspace Name",
#     "library:datasetVersion": "1.1",
#     "library:datasetDescription": "Cohort Description",
#     "library:datasetCustodian": "Sam Sneed",
#     "library:datasetDepositor": "Larry Sneed",
#     "library:contactEmail": "sam@sneed.com",
#     "library:datasetOwner": "Virgina Sneed",
#     "library:institute": ["Scool of Hard Knocks"],
#     "library:indication": "Poor health",
#     "library:numSubjects": 123,
#     "library:projectName": "TopMed",
#     "library:dataCategory": ["Simple Nucleotide Variation (SNVs), CNV"],
#     "library:datatype": ["Whole Exome", "Whole Genome"],
#     "library:dataUseRestriction": "HMB-ABC-123",
#     "library:studyDesign": "Case/Control",
#     "library:requiresExternalApproval": False,
#     "library:useLimitationOption": "skip",
#     "library:GRU": True,

# }   




    
getWorkspaceAttributes(CATALOG_WORKSPACE_NAMESPACE,TEST_WORKSPACE)

# updateAttributes(CATALOG_WORKSPACE_NAMESPACE,TEST_WORKSPACE,testAttributes)

200
# {"attributes":{"library:requiresExternalApproval":false,"library:discoverableByGroups":{"itemsType":"AttributeValue","items":["GP_DataDelivery_TaskTeam"]},"library:dulvn":4,"library:GR":true,"library:studyDesign":"Case/Control","description":"ANVIL_CMG_BROAD_BRAIN_ENGLE_WES","library:published":true,"library:indication":"Poor health","library:contactEmail":"sam@sneed.com","library:numSubjects":123,"library:datasetOwner":"Virgina Sneed","library:datatype":{"itemsType":"AttributeValue","items":["Whole Exome","Whole Genome"]},"library:datasetCustodian":"Sam Sneed","library:projectName":"TopMed","library:institute":{"itemsType":"AttributeValue","items":["Scool of Hard Knocks"]},"tag:tags":{"itemsType":"AttributeValue","items":["Access WS","DUL:GSR:notAllowed","GRCh38/hg38","Consortium: CMG","dbGaP: phs001272"]},"library:dataUseRestriction":"HMB-ABC-123","library:datasetDepositor":"Larry Sneed","library:datasetVersion":"1.1","library:datasetName":"Workspace Name","library:dataCategory":{"itemsType":"AttributeValue","items":["Simple Nucleotide Variation (SNVs), CNV"]},"library:useLimitationOption":"skip","library:datasetDescription":"Cohort Description"},"authorizationDomain":[{"membersGroupName":"AUTH_AnVIL_CMG_Broad_Brain_Engle_WES"}],"billingAccount":"billingAccounts/00C5C3-F6B804-1667EC","bucketName":"fc-secure-76700a19-3005-4f2e-a595-abed41d306d4","completedCloneWorkspaceFileTransfer":"2022-03-04T22:43:02.100Z","createdBy":"dave@clevercanary.com","createdDate":"2022-03-04T22:42:55.280Z","googleProject":"terra-51086c51","googleProjectNumber":"568298150090","isLocked":false,"lastModified":"2022-03-04T22:42:56.769Z","name":"Clever_Canary_ANVIL_CMG_BROAD_BRAIN_ENGLE_WES_TEST","namespace":"anvil-datastorage","workflowCollectionName":"76700a19-3005-4f2e-a595-abed41d306d4","workspaceId":"76700a19-3005-4f2e-a595-abed41d306d4","workspaceVersion":"v2"}




401


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head><script src="https://us.jsagent.tcell.insight.rapid7.com/tcellagent.min.js" tcellappid="FCProd-EBOgM" tcellapikey="AQQBBAFLGLOxL7VE9IF9ESlLvCxD5Ykr_7xkQKq_rgn_P58IWjOhOzIh6p3aI4pTWaprlUw" tcellbaseurl="https://us.agent.tcell.insight.rapid7.com/api/v1"></script>
<title>401 Unauthorized</title>
</head><body>
<h1>Unauthorized</h1>
<p>This server could not verify that you
are authorized to access the document
requested.  Either you supplied the wrong
credentials (e.g., bad password), or your
browser doesn't understand how to supply
the credentials required.</p>
<hr>
<address>Apache Server at api.firecloud.org Port 443</address>
</body></html>
: 0